In [1]:
import qe.myqengine as qe
import pandas as pd
import numpy as np

In [2]:
time_series = qe.TimeSeries(start_date = '0M',end_date = '0M', frequency='M',calendar='NAY')

univ = qe.IdUniverse(ids = ['AAPL-US','META-US','AMZN-US','NVDA-US','GOOGL-US','MSFT-US','TSLA-US'],universe_type='Equity')
data_dict = {"Broker Name":"FE_BROKER_ESTIMATE(SNAP,EPS,BKR_NAME,ANN_ROLL,+1,#DATE,,',AWIN=1D,ESTDATE=INPUT,WIN=31,')",
             'Analyst Name':"FE_BROKER_ESTIMATE(SNAP,EPS,AN_NAME,ANN,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT,')",
             "EST_EPS_FY1":"FE_BROKER_ESTIMATE(SNAP,EPS,EST_VALUE,ANN,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT')",
             "PREV_EST_EPS_FY1":"FE_BROKER_ESTIMATE(SNAP,EPS,PREV_VALUE,ANN,+1,#DATE,,'WIN=31,AWIN=1D,ESTDATE=INPUT')",
             "Research Date":"FE_BROKER_ESTIMATE_DATE(SNAP,EPS,INPUTDATETIME,ANN,+1,'YYYYMMDD',#DATE,,'AWIN=1D,ESTDATE=INPUT,WIN=31')",
             "Analyst ID":"FE_BROKER_ESTIMATE(SNAP,EPS,AN_ENTITY_ID,ANN,+1,#DATE,,',AWIN=1D,ESTDATE=INPUT,WIN=31,')",
             "Cumulative Return":"P_PRICE_RETURNS(1,20220831,#DATE,RANGE,USD)", 
             "Report Date":"FE_ESTIMATE_DATE(RPT_DATE,,QTR_ROLL,+1,'YYYYMMDD',#DATE,,,'')",
             "Surprise":"FE_SURPRISE(PERCENT,EPS,MEAN,QTR_ROLL,0,#DATE,,,'')",
             }
calc = qe.Calculation(universe = univ, dates = time_series,data_dict=data_dict,
                                        source = 'FqlExpression',
                                        is_array=True)
req = calc.query()

df = req.data.rename(columns={'UNIVERSE':"Ticker","DATE":'Date'}).set_index("Ticker")

df['Date'] = pd.to_datetime(df['Date']).apply(lambda x:x.strftime("%m/%d/%Y"))
df['Cumulative Return']= df['Cumulative Return'].apply(lambda x:x[0])
df['Report Date']= pd.to_datetime(df['Report Date'].apply(lambda x: x[0]))
df['Surprise']= df['Surprise'].apply(lambda x: x[0]).astype(float)
df_e = pd.concat([df['Analyst ID'].explode().reset_index()[['Ticker','Analyst ID']],df['Broker Name'].explode().reset_index().iloc[:,-1:],df['Analyst Name'].explode().reset_index().iloc[:,-1:],df['EST_EPS_FY1'].explode().reset_index().iloc[:,-1:],df['PREV_EST_EPS_FY1'].explode().reset_index().iloc[:,-1:],df['Research Date'].explode().reset_index().iloc[:,-1:]],axis=1)
df_e['Research Date'] = pd.to_datetime(df_e['Research Date'])
df_e = df_e.replace('Restricted',np.nan).replace('@NA',np.nan)
df_e = df_e.groupby(['Research Date','Broker Name','Ticker']).agg({'EST_EPS_FY1':'mean'})

df_e.head()

EST_EPS_FY1
Research Date Broker Name               Ticker              
2023-08-02    KeyBanc Capital Markets   MSFT-US    10.930000
              Needham                   META-US    12.280000
2023-08-03    Credit Suisse             AAPL-US     6.066816
              Daiwa Securities Co. Ltd. META-US    13.790000
              Jefferies                 AMZN-US     2.130000